# Dashboarding (with Dash & Streamlit)

- online-ds-ft-081720
- 01/05/20

## Learning Objectives

- Developing a Dash app inside a Jupyter Notebook first with `JupyterDash`.

- Become familiar with the fundamentals of making a Dash app.
    - Learn The Required Imports and what they contain
        - Dash HTML
        - Dash Core Components
    - Learn how to set up 
    - Callbacks
    


- Walk Through Basic 
- Walk-through using an API (Yahoo Finance stock prices via Pandas DataReader) to update dashboard data based on user input

# Installing `Dash` and Our First Tutorial App

- Check Dash's documentation for the installation code. 
    - https://dash.plotly.com/installation
        - As of 01/04/20: 
            - `pip install dash==1.18.1`
        
> NOTE: You may need to install dash in your base Conda Environment if you get an error about no package named dash. To do so, just deactivate learn-env and then run the pip install command again.

- To Switch to Your Base Environment in your terminal:
    - On Mac:
    ```bash
    ## Deactivate Learn-Env
    conda deactivate
    ## Install Dash
    pip install dash==1.18.1
    ## Then reactivate learn-env
    conda activate learn-env
    ```
    - On Windows:
    ```bash
    ## Deactivate Learn-Env
    source deactivate
    ## Install Dash
    pip install dash==1.18.1
    ## Then reactivate learn-env
    source activate learn-env
    ```

In [ ]:
import dash
dash.__version__

## Our First Basic App

- Normally, Dash apps live inside of .py files (traditionall `app.py`) and we run them from our terminal with `python app.py`
- The app then boots up and you can view it by navigating the displayed address  (usually http://127.0.0.1:8050/ )

### App Layout
- Copy the [Dash Tutorial's app layout](https://dash.plotly.com/layout) as a starting point (see below)
- Let's first use it in an external .py file `for_class/tutorial_app.py`
- Open a new terminal and navigate to the folder with the tutorial_app.py file. 
- Then run: `python tutorial_app.py`
- Open the displayed URL (should start with 127...) to see the app!

> Open and run `for_class/tutorial_app.py`

# Using `JupyterDash` to Develop a Dash App in Your Notebook

- To develop your apps inside your notebook instead of inside a .py file, use `JupyterDash`!

- [`JupyterDash`is a package](https://github.com/plotly/jupyter-dash) that lets us run and display Dash apps inside our Jupyter notebook.
- To Install:
`pip install jupyter_plotly_dash`

#### There are 2 major changes we have to make:
1. All we have to do is change how we initialize our app:
```python 
import dash
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)
```
becomes 
```python
from jupyter_plotly_dash import JupyterDash
app = JupyterDash('our app')
```
    - Unfortunately CSS styling is not supported in JupyterDash, so for now we will exclude the external_stylesheets argument. 

- And the `if __name__ == '__main__'` code at the bottom of the .py file:
```python
if __name__ == '__main__':
    app.run_server(debug=True)
```
becomes just 
```python 
app
```

In [ ]:
# !pip install -U jupyter_plotly_dash
from jupyter_plotly_dash import JupyterDash

In [ ]:
# -*- coding: utf-8 -*-

# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd

## OLD CODE
# external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
# app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

## NEW CODE
from jupyter_plotly_dash import JupyterDash
app = JupyterDash('our app')


# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for Python.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

## OLD CODE
# if __name__ == '__main__':
#     app.run_server(debug=True)
    
    
## NEW CODE

app

# Making Our Own App for Displaying Stock Data

## Importing and Using Plotly, Pandas DataReader

In [ ]:
## General Imports
import numpy as np
np.random.seed(321)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (12,4)

### Pandas DataReader

In [ ]:
## Pandas DataReader Import
# %conda install -c anaconda pandas-datareader
import pandas_datareader as pdr

In [ ]:
## Get GOOGL stock price for 2012-2020 from Yahoo Finance
example = pdr.DataReader('GOOGL','yahoo','2012','2020')
example

In [ ]:
example.plot(y='Adj Close')

### Plotting with Plotly Express

- Plotly Express Overview:
    - https://plotly.com/python/plotly-express/

In [ ]:
## Plotly Imports
import plotly.express as px
import plotly.io as pio
pio.templates

In [ ]:
## Setting Plotly Theme
pio.templates.default = "seaborn"

In [ ]:
pfig = px.scatter(example,y='Adj Close',title='GOOGL')
pfig

### Extending to Multiple Stocks

In [ ]:
## List of symbols to grab
symbols = ['FB','AAPL','GOOGL','AMZN','MSFT']
data = {}
for stock in symbols:
    data[stock] = pdr.DataReader(stock,'yahoo','2012','2021')['Adj Close']
df = pd.DataFrame(data)#.reset_index()
df

In [ ]:
## PLotting with PLotly Express
pfig = px.scatter(df,y=['FB','GOOGL'])
pfig

### Functionizing Loading Data + Plotting

In [ ]:
import datetime as dt
import time

today = dt.date.today()
def get_data(start_date='2012-02-01',end_date=today,
             symbols = ['FB','AAPL','GOOGL','AMZN','MSFT','TSLA']):
    """ Gets the historical stock dataa for the provided symbols for the 
    time period requested 
    """
    data = {}
    for stock in symbols:
        time.sleep(0.2)
        try:
            data[stock] = pdr.DataReader(stock,'yahoo',start_date,end_date)['Adj Close']
        except:
            print('Error with stock: '+stock)
    df = pd.DataFrame(data)#.reset_index()
    return df

In [ ]:
df = get_data(start_date='2016-03-01',end_date=today)
df

In [ ]:
def plot_stocks(df,stocks=[]):
    if len(stocks)==0:
        stocks=None
    pfig = px.scatter(df,y=stocks)
    return pfig
    
fig = plot_stocks(df)
fig

In [ ]:
## PLotting 2 of the available cols   
fig = plot_stocks(df,['FB','MSFT'])
fig

## Making Our First Stock Data App

- For our first app we will get the data for a larger number of stock symbols
- We will then provide a Dropdown box for the user to select which stocks they would like to see.

In [ ]:
## getting S&P500 symbols
import requests,bs4
url= 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
response = requests.get(url)
response.status_code

In [ ]:
## Get tables from url
result = pd.read_html(response.content)
print(type(result))
print(len(result))
result[0]

In [ ]:
## Gettting S&P500
sp500 = result[0].copy()
sp500.head()

In [ ]:
## Let's grab just one sector 
sp500['GICS Sector'].value_counts()

In [ ]:
## Pick a sector to use
sector='Communication Services'
stocks_to_get = sp500.loc[ sp500['GICS Sector'] == sector,
                          'Symbol'].to_list()
stocks_to_get

In [ ]:
len(stocks_to_get)

In [ ]:
## Getting the data FIRST


### Constructing our App

In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html

- `dash_core_components` (dcc) contains widgets.
    - We will use `dcc.Dropdown` to select stocks to display
- `dash_html_components` (html) contains HTML tags as objects. (Div, H1, etc.)
    - We will use `html` for the layout of our app.

In [ ]:
## App here



## Our App with Dropdown

- The Dropdown menu expects a list of dictionaries for the options to display. 
    - Each element of the list gets a `'label'` (what to show the user) and a `'value'` (what the actual data is named).
    
    
```python

stock_options= [{'MSFT':'MSFT'},{'AMZN':'AMZN'}]
```


In [ ]:
def make_options(menu_choices,ignore_cols=['Date']):
    """Returns list of dictionary with {'label':menu_choice,'value':menu_choice}"""
    options = []
    for choice in menu_choices:
        if choice not in ignore_cols:
            options.append({'label':choice,'value':choice})
    return options

make_options(df)

In [ ]:
## App here


> #### It is still showing ALL of the data. Why? <br> Because we need Callbacks!


# Dash Callbacks

**Callbacks are functions that handle the interactivity between elements of our app**

>#### Callback Documentation\
- https://dash.plotly.com/basic-callbacks
- https://dash.plotly.com/advanced-callbacks

## Basic Callbacks

### To write a callback you will need: 

1. Additional imports from dash.dependencies
```python
from dash.dependencies import Input, Output
```
2. To make sure all components that need to connect together to have an `id`
```python
dcc.Dropdown(id='choose_stocks',...
         ```
- A function that will update the visuals displayed
```python
def update_stocks(stocks):
    return plot_stocks(df,stocks=stocks)
```
- A decorator function above that which uses the `Input` and `Output` classes to connect the arguments of your function with the pieces of the app that contain the information needed.

```python
@app.callback(Output('line_plot','figure'),
              [Input('choose_stocks','value')])
def update_stocks(stocks):
    return plot_stocks(df,stocks=stocks)
```

In [ ]:
## App Here

## More Advanced Callbacks

- Multiple Inputs
- Using State to delay processing updates

### Adding Date Selection
- https://dash.plotly.com/dash-core-components/datepickerrange

In [ ]:
import datetime as dt
from datetime import date
import pandas_datareader as pdr

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import numpy as np
np.random.seed(321)

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['figure.figsize'] = (12,4)

import plotly.express as px
import plotly.io as pio
pio.templates


today = dt.date.today().strftime("%Y-%m-%d")
def get_data(start_date='2012-02-01',end_date=today,symbols = ['FB','AAPL','GOOGL','AMZN','MSFT']):
    
    data = {}
    for stock in symbols:
        try:
            data[stock] = pdr.DataReader(stock,'yahoo',start_date,end_date)['Adj Close']
        except:
            print('Error with stock: '+stock)
    df = pd.DataFrame(data).reset_index()
    return df


def plot_stocks_df(df=None,stocks=['FB','AAPL']):
    if df is None:
        df = get_data(stocks)#.reset_index()
    stocks_exist = [s for s in stocks if s in df.columns]
    pfig = px.scatter(df,x='Date',y=stocks_exist)
    return pfig

from jupyter_plotly_dash import JupyterDash
app = JupyterDash('Example', 
                  serve_locally=False)
defaults = ['MSFT','AMZN']
# df = get_data(date(2012, 1, 1))

app.layout = html.Div(children = [
    html.H1('Our AMAZING Dashboard'),
    
    html.Div(children=[
        html.Div(style={"border":'1px solid black','padding':'3px'},id='menu',children=[
            html.H3('Enter stock symbols, separated by a comma.\nPress Enter to update.'),

            dcc.Input(id='choose_stocks',value='MSFT,AMZN',
                      placeholder='MSFT,AAPL',#,
                         style={'width': '90%', 'height': 50}),           

            dcc.DatePickerRange( id='my-date-picker-range',
                                start_date=date(2012,1,1),
                                min_date_allowed=date(2010, 1, 1),
                                max_date_allowed=today,#date.today(),#date(2021),
                                end_date=today,style={'width':'50%'}),#date.today()), #date(2017, 8, 25)),
            html.Button('Submit',id='submit',style={'size':'40 px'})
    
        ]),
        
        dcc.Graph(id='line_plot')

    ])
])

@app.callback(Output('line_plot','figure'),
              [Input('submit','n_clicks')],
              [State('choose_stocks','value'),
               State('my-date-picker-range', 'start_date'),
               State('my-date-picker-range', 'end_date')])
def update_stocks(n_clicks, stocks,start_date,end_date):
    stocks = stocks.split(',')
    plot_df = get_data(start_date,end_date,symbols=stocks)
    return plot_stocks_df(df=plot_df,stocks=stocks)#,start_date,end_date)
    
app

# Streamlit

- Streamlit is a new, easy-to-use package for developing apps.
    - https://www.streamlit.io/
    
- It is easier to use than Dash, but not as customizable. 



> Open `for_class>streamlit_app.py` and compare!

In [ ]:
from IPython.display import display,Markdown
with open('for_class/streamlit_app.py') as file:
    display(Markdown("```python\n"+file.read()+'\n```'))

# Appendix / If There's Time 

## Deploying Your App For Free with Heroku


1. Go through official Heroku Python Tutorial:
    - https://devcenter.heroku.com/articles/getting-started-with-python
    
    
    
2. Use this template as a starting point for your app.
    - ~~https://github.com/plotly/dash-heroku-template~~
    - My Modified Template:
        - https://github.com/jirvingphd/dash-heroku-template.git
    
    
3. Go through official Dash tutorial to get your app.py started with their example Layout app
    - https://dash.plotly.com/
    
4. Checkout the Official Dash/Herokuapp Tutorial for how to make your repo into an app 
    - https://dash.plotly.com/deployment
  